In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
A1_cost = 3.25
B1_cost = 3.40
A_r = 100000
B_r = 30000
R = 100000
R_cost = 0.35

#snd article

A_r1= 0.15
A_r2 = 0.85
B_r1 = 0.25
B_r2 = 0.75

Conv = 50000
Conv_c = 0.25
Conv_r1 = 0.5
Conv_r2 = 0.5

Filt_c = 0.10


#sell

p2_c = 3.80
#p1_c45 = 5.50
#p1_c4 = 5.2
p1_c_inf = 5.0

p1_need = 40000

p1_b = 5.75
p2_b = 4.00




In [3]:
a = gp.Model('smth')
Q1 = a.addVar(vtype = GRB.INTEGER, name = 'buyA_before')
Q2 = a.addVar(vtype = GRB.INTEGER, name = 'buyB_before')
Q3 = a.addVar(vtype = GRB.INTEGER, name = '2to1_convert')
#Q4 = a.addVar(vtype = GRB.INTEGER, name = '1for_5.5')
#Q5 = a.addVar(vtype = GRB.INTEGER, name = '1for_5.2')
Q6 = a.addVar(vtype = GRB.INTEGER, name = 'buy1_after')
Q7 = a.addVar(vtype = GRB.INTEGER, name = 'buy2_after')
Q8 = a.addVar(vtype = GRB.CONTINUOUS, name = 'Price_of1')

Academic license - for non-commercial use only - expires 2022-09-10


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd0 in position 28: invalid continuation byte

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "src\gurobipy\extern.pxi", line 19, in gurobipy.__bytestostring
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd0 in position 28: invalid continuation byte


In [4]:
pr1_amount = A_r1*Q1 + B_r1*Q2 + Conv_r1*Q3 + Q6
pr2_amount = A_r2*Q1 + B_r2*Q2 - Conv_r2*Q3 + Q7

charges = A1_cost*Q1 + B1_cost*Q2 + R_cost*(Q1 + Q2) + Conv_c*Q3 +\
Filt_c*(A_r1*Q1 + Conv_c*Q2 + Conv_r1*Q3) + p1_b*Q6 + p2_b*Q7

income = pr1_amount*Q8 + p2_c*pr2_amount

a.setObjective(Q8, GRB.MINIMIZE)

In [5]:
a.addConstr(Q1<=A_r)
a.addConstr(Q2<=B_r)
a.addConstr(Q1+Q2<=R)
a.addConstr(Q3<=Conv)
a.addConstr(pr1_amount>=p1_need)
a.addConstr (Q3<=A_r2*Q1 + B_r2*Q2 + Q7)
a.addConstr(income - charges>=50000)
a.params.NonConvex =2 
a.update()

print(a.display())
a.optimize()

Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Minimize
   <gurobi.LinExpr: Price_of1>
Subject To
   R0 : <gurobi.LinExpr: buyA_before> <= 100000.0
   R1 : <gurobi.LinExpr: buyB_before> <= 30000.0
   R2 : <gurobi.LinExpr: buyA_before + buyB_before> <= 100000.0
   R3 : <gurobi.LinExpr: 2to1_convert> <= 50000.0
   R4 : <gurobi.LinExpr: 0.15 buyA_before + 0.25 buyB_before + 0.5 2to1_convert + buy1_after> >= 40000.0
   R5 : <gurobi.LinExpr: -0.85 buyA_before + -0.75 buyB_before + 2to1_convert> <= 0.0
    : <gurobi.QuadExpr: -0.385 buyA_before + -0.9250000000000003 buyB_before + -2.1999999999999997 2to1_convert + -5.75 buy1_after + -0.20000000000000018 buy2_after + [ 0.15 buyA_before * Price_of1 + 0.25 buyB_before * Price_of1 + 0.5 2to1_convert * Price_of1 + buy1_after * Price_of1 ]> >= 50000.0
General Integers
   ['buyA_before', 'buyB_before', '2to1_convert', 'buy1_after', 'buy2_after']
None
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread 

In [6]:
if a.status == GRB.OPTIMAL:
    print('\nProce of p1: %g' % a.objVal)
    for v in a.getVars():
        print('%s %g' % (v.varName, v.x))
else:
    print('No solution')


Proce of p1: 4.9625
buyA_before 100000
buyB_before 0
2to1_convert 50000
buy1_after -0
buy2_after 0
Price_of1 4.9625
